In this notebook, I aim to roll through an analysis across a few patients which can easily be extended for all of the  patients in your cohort. To do so, we will use the pre-processing functions that are written out more explicitly in the step-by-step notebooks. 

**This is the notebook you should copy and edit for your own actual analyses**

======================================================================================================================

These are magics that provide certain functionality. Specifically, if you edit functions that are called in this notebook, the functions are reloaded so the changes propagate here without needing to reload the notebook.


In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import pickle
from tqdm import tqdm
from IPython.display import clear_output
import warnings 

# I only want to see warnings once
warnings.filterwarnings('ignore')

Note: If you have installed the LFPAnalysis package in editable form on Minerva, you must append the local path! This is because Minerva requires that you point your package installs away from the local directory for space reasons, but editable packages have to be installed locally.

In [4]:
import sys
sys.path.append('/hpc/users/qasims01/resources/LFPAnalysis')

In [5]:
from LFPAnalysis import lfp_preprocess_utils, sync_utils, analysis_utils, nlx_utils

## Load, pre-process and re-reference the neural data

Substitute your own patients in the following cell: 

In [7]:
# Specify root directory for un-archived data and results 
base_dir = '/sc/arion' 
# Specify subject names 
subj_ids = ['MS012', 'MS016']
subj_sites = ['MSSM', 'MSSM']
# Specify the format being loaded. Probably smart to put subjects and their format in a dataframe. 
subj_formats = ['edf', 'edf']
elec_dict = {f'{x}': [] for x in subj_ids}
mne_dict = {f'{x}': [] for x in subj_ids}
photodiode_dict = {f'{x}': [] for x in subj_ids}

## Pre-process (run 1x): 

In the pre-processing functions below, we: 

1. load the raw data (either a .edf file or a folder of .nlx files) into mne objects for use with the mne toolbox: https://mne.tools/stable/index.html.

2. load the localized electrode names from the .csv or .xlsx file listing their MNI coordinates into the mne object

3. filter and resample as necessary

4. re-reference to a proximal white matter electrode (or bipolar, if desired) 

In [1]:
for ix, subj_id in enumerate(subj_ids): 
    site = subj_sites[ix]
    format = subj_formats[ix]
    
    print(f'Working on subj {subj_id}')
    
    # Set paths
    load_path = f'{base_dir}/projects/guLab/Salman/EMU/{subj_id}/neural/Day1'
    elec_path = f'{base_dir}/projects/guLab/Salman/EMU/{subj_id}/anat/'
    save_path = f'{base_dir}/projects/guLab/Salman/EphysAnalyses/{subj_id}/neural/Day1'
    
    # Check if path exists for saving, and if not, make it
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # electrode files could either be csv or excel
    elec_files = glob(f'{elec_path}/*.csv') + glob(f'{elec_path}/*.xlsx')
    # There should really only be one, so grab it with the zero-index 
    elec_file = elec_files[0]

    # Make MNE file
    mne_data = lfp_preprocess_utils.make_mne(load_path=load_path, 
                                             elec_path=elec_file,
                                             format=format,
                                             return_data=True,
                                             site=site,
                                             check_bad=False) # changed this to not annotate anything as bad 

    # Save this data so that you don't need this step again:
    mne_data.save(f'{save_path}/raw_ieeg.fif', overwrite=True)


## DEPRECATED: ~Manually annotate bad channels before re-referencing~

~Even though the pipeline automatically detects channels with high variance compared to the rest of the data, it can still be very useful to scroll through the data and manually click on channels that are bad (or click on automatically bad channels that are,in fact, not bad).~

~Feel free to be liberal with this. Bad channels are not omitted from analysis; they are simply omitted from being used as references, so they don't spread the 'badness' to other channels.~


~We want to identify channels that are CLEARLY artifactual. I say CLEARLY because there are some noisy channels that will clear up with re-referencing. We want to pick channels here only if they are clear outliers that should not be USED as references for any other channels - this would simply spread their noise around.~" 

Note: Viewing/scrolling/editing these channels requires Jupyter NOTEBOOK - won't work in JupyterLab. 

In [2]:
# %matplotlib notebook 
# # Note: this does not work in JupyterLAB.

In [1]:
# # Scroll up/down and left/right using your keyboard. CLICK on a channel to turn it 'grey' and mark as a 'bad' channel. 
# # If you click a grey channel again it will unmark it. 

# subj_id = 'MS012'
# save_path = f'{base_dir}/work/qasims01/MemoryBanditData/EMU/Subjects/{subj_id}/neural/Day1'
# mne_data = mne.io.read_raw_fif(f'{save_path}/raw_ieeg.fif', preload=True)
# fig = mne_data.plot(start=0, duration=120, n_channels=30, 
#                       scalings=mne_data._data.max()/30
#                      )

In [3]:
# # Save this manual data
# mne_data.save(f'{save_path}/raw_ieeg.fif', overwrite=True)

## Re-reference the data (default=bipolar): 

We re-reference the data to get rid of shared noise, cleaning the data to leave what we assume is local biological activity. 

In [ ]:
for ix, subj_id in enumerate(subj_ids): 
    site = subj_sites[ix]
    format = subj_formats[ix]

    print(f'Working on subj {subj_id}')
    
    # Set paths
    load_path = f'{base_dir}/projects/guLab/Salman/EMU/{subj_id}/neural/Day1'
    elec_path = f'{base_dir}/projects/guLab/Salman/EMU/{subj_id}/anat/'
    save_path = f'{base_dir}/projects/guLab/Salman/EphysAnalyses/{subj_id}/neural/Day1'
    # Check if path exists for saving, and if not, make it
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # electrode files could either be csv or excel
    elec_files = glob(f'{elec_path}/*.csv') + glob(f'{elec_path}/*.xlsx')
    # There should really only be one, so grab it with the zero-index 
    elec_file = elec_files[0]

    # Make MNE file
    mne_data = mne.io.read_raw_fif(f'{save_path}/raw_ieeg.fif', preload=True)


    # Re-reference neural data
    mne_data_reref = lfp_preprocess_utils.ref_mne(mne_data=mne_data, 
                                                  elec_path=elec_file, 
                                                  method='bipolar', 
                                                  site=site)

    # Save this data so that you don't need this step again:
    mne_data_reref.save(f'{save_path}/bp_ref_ieeg.fif', overwrite=True)

    # Should also save out re-referenced elec_file: 

    elec_data = lfp_preprocess_utils.load_elec(elec_file)
    anode_list = [x.split('-')[0] for x in mne_data_reref.ch_names]
    elec_df = elec_data[elec_data.label.str.lower().isin(anode_list)]
    elec_df['label'] =  elec_df.label.apply(lambda x: [a for a in mne_data_reref.ch_names if str(x).lower() in a.split('-')[0]][0])

    # add region to the data frame 
    manual_col = [col for col in elec_df.columns if 'manual' in col.lower()][0]
    all_regions = [] 
    for chan_name in elec_df.label.unique():
        elec_region = analysis_utils.select_rois_picks(elec_df, chan_name, manual_col=manual_col)
        all_regions.append(elec_region) 

    elec_df['salman_region'] = all_regions
    elec_df['hemisphere'] = elec_df.label.apply(lambda x: x[0])

    elec_df.to_csv(f'{base_dir}/projects/guLab/Salman/EphysAnalyses/{subj_id}/reref_elec_df', index=False)
            


 - mne_data: a Raw mne object, where the data has been loaded, filtered for line noise, searched for bad channels, parsed for different data types, and resampled if necessary. 
 
 - mne_data_reref: an mne object containing re-referenced data (either white matter or bipolar)

## NOW look at the data to manually remove channels: 

After bipolar referencing: 

In [ ]:
%matplotlib notebook 

In [ ]:
# Scroll up/down and left/right using your keyboard. CLICK on a channel to turn it 'grey' and mark as a 'bad' channel. 
# If you click a grey channel again it will unmark it. 

subj_id = 'MS012'
save_path = f'{base_dir}/projects/guLab/Salman/EphysAnalyses/{subj_id}/neural/Day1'
mne_data = mne.io.read_raw_fif(f'{save_path}/bp_ref_ieeg.fif', preload=True)
fig = mne_data.plot(start=0, duration=120, n_channels=30, 
                      scalings=mne_data._data.max()/30
                     )

In [ ]:
# ALSO look at the power spectra! 
# You can click on channels here to identify them, and go back to the viz above to mark them as noise if need be

mne_data_reref.compute_psd().plot()

If you have ran the preprocessing above, load the data instead: 


In [8]:
for subj_id in subj_ids: 
    # Set paths
    load_path = f'{base_dir}/projects/guLab/Salman/EMU/{subj_id}/neural/Day1'
    elec_path = f'{base_dir}/projects/guLab/Salman/EMU/{subj_id}/anat/'
    save_path = f'{base_dir}/work/qasims01/MemoryBanditData/EMU/Subjects/{subj_id}/neural/Day1'

    elec_files = glob(f'{elec_path}/*.csv') + glob(f'{elec_path}/*.xlsx')
    # There should really only be one 
    elec_file = elec_files[0]
    elec_data = lfp_preprocess_utils.load_elec(elec_file)

    mne_data_reref = mne.io.read_raw_fif(f'{save_path}/bp_ref_ieeg.fif', preload=True)

    photodiode_data = mne.io.read_raw_fif(f'{load_path}/photodiode.fif', preload=True)

    # Append to list 
    mne_dict[subj_id].append(mne_data_reref)

    photodiode_dict[subj_id].append(photodiode_data)

    elec_dict[subj_id].append(elec_file)

/tmp/ipykernel_176324/4234602883.py:14: RuntimeWarning: This filename (/sc/arion/projects/guLab/Salman/EMU/MS012/neural/Day1/photodiode.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  photodiode_data = mne.io.read_raw_fif(f'{load_path}/photodiode.fif', preload=True)
/tmp/ipykernel_176324/4234602883.py:14: RuntimeWarning: This filename (/sc/arion/projects/guLab/Salman/EMU/MS016/neural/Day1/photodiode.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  photodiode_data = mne.io.read_raw_fif(f'{load_path}/photodiode.fif', preload=True)
/tmp/ipykernel_176324/4234602883.py:14: RuntimeWarning: This filename (/sc/arion/projects/guLa

 - mne_dict: a dictionary containing all of your subjects' re-referenced mne data 
 
 - photodiode_dict: a dictionary containing all of your subjects' photodiode data 
 
 - elec_dict: a dictionary containing the paths to your subjects' electrode data 

## Sync behavioral and neural data

Here, we perform a critical step: computing the time offset between the computer that recorded the neural data and the laptop that featured the experiment. 

The function here only requires a **subset** of detected sync signals (i.e. photodiode deflections) to be detected to successfully compute this offset. 

First, you may need to MANUALLY clean the photodiode signal if the recording quality is poor. Load it, plot it, and try to isolate/amplify the pulses. 

In [ ]:
subj_id = 'MS015'
temp_diode = photodiode_dict[subj_id][0]._data[0, :]
temp_diode[900000:] = np.nanmin(temp_diode)
photodiode_dict[subj_id][0]._data = temp_diod

In [12]:
slopes = {f'{x}': [] for x in subj_ids}
offsets = {f'{x}': [] for x in subj_ids}

for subj_id in subj_ids: 
        
    # Load the behavioral timestamps: 
    behav_path = f'{base_dir}/projects/guLab/Salman/EMU/{subj_id}/behav/Day1'
    temp_df = pd.read_csv(glob(f'{behav_path}/*.csv')[0], index_col=None, header=0)
    # Load in the timestamps pertaining to your sync. If your task had a square pop up, for example, grab the times for that square's appearance from the behavioral logs.
    # Below, I do this for my own task's Psychopy output, but yours is probably coded differently. 
    beh_ts = temp_df[temp_df.keys()[temp_df.keys().str.startswith('sync') & temp_df.keys().str.endswith('started')]].values
    beh_ts = beh_ts[~np.isnan(beh_ts)] 

    # Synchronize to the photodiode or whatever your neural sync signal is
    height = 1
    windSize = 15
    smoothSize = 11

    slope, offset = sync_utils.synchronize_data(beh_ts, 
                                                photodiode_dict[subj_id][0], 
                                                smoothSize=smoothSize, windSize=windSize, height=height)

    print(slope)
    print(offset)
    slopes[subj_id].append(slope)
    offsets[subj_id].append(offset)

28 blocks
............................

found matches for 180 of 428 pulses
28 blocks
............................

found matches for 345 of 426 pulses
31 blocks
...............................

found matches for 300 of 469 pulses


 - slopes: a dictionary containing the slopes (should be ~ 1) for each subject
 - offsets: a dictionary containing the offsets for each subject

## Load your behavioral data

You probably have a separate notebook for processing the behavioral data for your task. Load the processed dataframe here:

In [2]:
behav_data = pd.read_csv(f'{base_dir}/work/qasims01/MemoryBanditData/EMU/learn_df.csv')

In [ ]:
# Filter just to the participants in this notebook
behav_data = behav_data[behav_data.participant.isin(subj_ids)]

In [15]:
behav_data.head(5)[['participant', 'feedback_start', 'baseline_start']]

,participant,feedback_start,baseline_start
0,MS012,243.239158,244.929025
1,MS012,248.344187,250.043187
2,MS012,254.083059,255.790670
3,MS012,258.148220,259.838892
4,MS012,261.943712,263.620631


## Make epochs

Make epochs and remove IEDs. Currently just doing this for one example period - when subjects receive feedback. 

Notes: 

- I also segment a baseline period for every event of interest. 

- I apply a buffer period of 1.0 seconds - this will be helpful when we compute spectrograms later. 

- The IED count for every channel is added to the epoch metadata

(I'm a little dumb, so my baseline is a fixation cross AFTER the trial, rather than before. A bit silly if you ask me.) 

In [3]:
# set some windows of interest 

buf = 1.0 # this is the buffer before and after that we use to limit edge effects for TFRs

IED_args = {'peak_thresh':5,
           'closeness_thresh':0.25, 
           'width_thresh':0.2}

# evs = ['gamble_start', 'feedback_start', 'baseline_start']
evs = {'gamble_start': [-1.0, 0.5],
       'feedback_start': [-0.5, 1.5],
       'baseline_start': [0, 0.75]}


# add behavioral times of interest 
for subj_id in subj_ids:
    # Set paths
    load_path = f'{base_dir}/projects/guLab/Salman/EMU/{subj_id}/neural/Day1'
    save_path = f'{base_dir}/projects/guLab/Salman/EphysAnalyses/{subj_id}/neural/Day1'

    epochs_all_evs = {f'{x}': np.nan for x in evs}
    for event in evs.keys():
        pre = evs[event][0]
        post = evs[event][1]
        fixed_baseline = None
        behav_times = learn_df[(learn_df.participant==subj_id)][event]

        # THE following function will now SAVE out dataframes that indicate IED and artifact time points in your data
        
        epochs = lfp_preprocess_utils.make_epochs(load_path=f'{save_path}/bp_ref_ieeg.fif', 
                                                  slope=slopes[subj_id][day][0], offset=offsets[subj_id][day][0], 
                                                  behav_name=event, behav_times=behav_times,
                                                  ev_start_s=pre, ev_end_s=post, buf_s=1, downsamp_factor=None, IED_args=IED_args, detrend=0)


        epochs_all_evs[event] = epochs
        epochs_all_evs[event].save(f'{save_path}/{event}-epo.fif', overwrite=True)


 - epochs_all_evs: dictionary containing the epochs for all of your subjects' re-referenced data

Plot and examine the epochs if you'd like:

In [73]:
# %matplotlib notebook
# fig = epochs_all_subjs_all_evs['MS007']['feedback_start'].plot(n_epochs=10, n_channels=10)

In [74]:
# # Need this following line to save the annotations to the epochs object 
# fig.fake_keypress('a')

## Where do I go from here? 

At this point, you've successfuly pre-processed your iEEG data and sliced it around your timepoints of interest. These epochs are going to be the currency for many of your subsequent analyses, so make sure you TRUST THEM before proceeding to the other notebooks for analyses. 

From here, you can move on to the:

1. FOOOF: a notebook for computing power-spectra across trials and fitting their peaks 

2. TFRPlotsAndStatistics: a notebook for computing time-frequency spectra (trial-level), and computing several different statistics or simply saving the data out in dataframes. 

3. OscillationDetection(BOSC): a notebook for computing sliding burst detection and saving the data out in dataframes 

4. TimeResolvedRegression: a notebook for computing regression analysis at each timepoint of a timeseries. TFR-extracted band power is used as example. 

5. ConnectivityAnalysis: a notebook for computing different synchrony measures between electrodes. 